In [ ]:
# deadline = 13 jan 2023

# Description
Devise an antiplagiarism software specialized on poems. For simplicity, assume that the software is able to detect sentences taken from "La Divina Commedia" by Dante Alighieri. The text of the poem can be downloaded from here. 





The software must detect sentences of a given size S in terms of words. Focus on the specific case S=6. 

Compare the performance and the memory occupancy of the anti-plagiarism system for S=6 based on:

a set storing the sentences
a set storing the fingerprints of the sentences 
a bit string array
a optimally-designed bloom filter
Preliminary study:

What are the inputs and the outputs of the software?
Evaluate the number of sentences that are stored for S=6. 
Evaluate the average size of each sentences in bytes.
Set of sentences:

What is the corresponding memory occupancy?
Fingerprint set:

By simulation, evaluate Bexp defined as the minimum value of bits such that no collisions are experienced when storing all the sentences in a fingerprint set.
Compute analytically the number of bit Bteo necessary to get a probability 0.5 of fingerprint collision when storing all the sentences with a fingerprint set.

Compute the probability of false positive for a fingerprint set in which the number of bits is Bexp, as computed in question 1.

Is Bteo a good approximation of Bexp?

Bit string array:

By simulation, evaluate the probability of false positive in function of memory X, for X in the interval [2^{19}, 2^{20}, 2^{21}, 2^{22}, 2^{23}] bits, and plot the corresponding graph.

By theory, evaluate the probability of false positive in function of X, and plot the corresponding graph.

Compare the simulation with the theoretical results.

How this solution compares with the fingerprint set above with Bexp bits?

Bloom filters:


Compute analytically the optimal number of hash functions in function of memory X, for X in the interval [2^{19}, 2^{20}, 2^{21}, 2^{22}, 2^{23}] bits, and plot the corresponding graph. 
By theory, evaluate the probability of false positive in function of X, using the optimal number of hash functions.
Describe how you implemented the different hash functions.
By simulation, evaluate the probability of false positive in function of X, using the optimal number of hash functions, and plot the corresponding graph.
Compare the simulation results with the results obtained by just analytical formulas. 
(optional) It can be shown that the number of distinct elements stored in a bloom filter with n bits and k hash functions can be estimated as  -\frac{n}{k}\ln \big(1-\frac{N}{n}\big)   where N is the actual number of bits equal to 1 in the bloom filter. Show how accurate is the formula when inserting one single sentence at the time in the bloom filter, in function of the inserted sentences.



Finally, discuss the different tradeoff between memory and performance achieved each data structure, based on the results above summarizied in the following table.

Data structure	Memory [kB]	Prob. false positive
Set of sentences		
Fingerprint set (X=Bexp)		
Bit string array (X varying)		
Bloom filter (X varying)		

What is the best option for this scenario?


In [ ]:
# I have to read the bloom filters pdf and also watch video for the firsrsession of 19th of December 

In [ ]:
# Q:what is the formula we should use for the analytical part?
#A: explained in the layers parts 
#when will he upload the video of the first session?
#A: He will upoload it but idk when
# by simulation: obvious, Analytically = formulas(maybe), theory?
# theory and analytically are the same 
#Slide 08/ page 16 is important (it’s what we have to keep in mind)
#It’s useful for set of fingerprints question 2 
# we should watch the video fro this session to be able to run the simulation (19th of December, session one)
# page 28 and 29 are important for the bloom filter 
# formula is page 28 gives us a real number and we have to either use 12 or 13, we have to choose the optimal value
# since bloom filters do not work with real numbers 
# for bloom filters, question 1 and two are different but somehow related
#the formula in page 29 is important but only the first part = (1-P) ** K (only this part is important)

In [1]:
import hashlib
import re
from pympler import asizeof
import math
import matplotlib.pyplot as plt


In [2]:
#############################################33
file = open("commedia.txt", "rt")
data = file.read()

In [4]:
#===========================
data=re.sub(r"[^\w\d'\s]+",'',data) # deleting the punctuations
words = data.split('\n')
del words[0:8]
clean_words=[]
for line in words.copy():
    if line.startswith("Inferno") or line.startswith("Purgatorio") or line.startswith("Paradiso")or not line.strip():
        words.remove(line) # deleting the headers 
    else:
        line = line.strip()
        splited_words = line.split(" ")
        clean_words.extend(splited_words)
print('the total number of words is \n')
print(len(clean_words)) #counting the number of words 
##############################################.

the total number of words is 

96929


In [5]:
asizeof.asizeof(clean_words)
#it's in bytes

6054136

In [6]:
print(f'number of distinct words is equal to {len(set(clean_words))}')

number of distinct words is equal to 14647


In [7]:
plagiarism_size= 4 #window size 
verses = []
finger_prints=[]
verse_count=0




for i in range (len(clean_words)):
    if len(clean_words[i:i+plagiarism_size])==4:
        verses.append(' '.join(clean_words[i:i+plagiarism_size]))

    
m = len(verses)
epsilon = 10 ** -4
b = math.log((m/epsilon) , 2 ) # b has to larger or equal to this value 
n = m * b


for i in range (len(verses)):
    temp=verses[i]
    encoded=temp.encode('utf-8')
    hashed=hashlib.md5(encoded)
    int_hashed=int(hashed.hexdigest(), 16)
    h = int_hashed % n
    finger_prints.append(h)


        
        
#we should store the fingerprints in a hashtable 
#then we should computer and store the finger prints inside of a hashtable 
print(f'\ntotal number of verses is {len(verses)}')



total number of verses is 96926


In [8]:
verses_set=set(verses)

In [9]:
finger_prints_set=set(finger_prints)

In [10]:
#it's better to have both fingerprints and the verses stored in a dictionary 
# so we're gonna do it from the beginning
new_verses=[]
# m = len(verses) 
# we already have the value for m , b , epsilon and n since we've ran the code before
# epsilon = 10 ** -4
# b = math.log((m/epsilon) , 2 ) # b has to larger or equal to this value 
# n = m * b
general_dict={}

for i in range (len(clean_words)):
    if len(clean_words[i:i+plagiarism_size])==4:
        new_verses.append(' '.join(clean_words[i:i+plagiarism_size]))
        temp=new_verses[i]
        encoded=temp.encode('utf-8')
        hashed=hashlib.md5(encoded)
        int_hashed=int(hashed.hexdigest(), 16)
        h = int_hashed % n
        finger_prints.append(h)
        if h in general_dict.keys():
            general_dict[h] + " , " + temp
        else:
            temp_dict={h:temp}
            general_dict.update(temp_dict)